# Transformers with PyTorch

##1. Loading Dataset

In [1]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.1 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

# Loading the rotten_tomatoes dataset:
dataset = load_dataset("rotten_tomatoes")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

## 2. Model Loading

In [3]:
from transformers import AutoModelForSequenceClassification

# Loading DistilBERT:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 3. Tokenization

In [4]:
from transformers import AutoTokenizer

# Loading tokenizer for DistilBERT:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# Creating a function to tokenize the dataset:
def tokenize_dataset(dataset):
    return tokenizer(dataset["text"])

# Applying tokenizer to all dataset:
dataset = dataset.map(tokenize_dataset, batched=True)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

## 4. Padding

In [6]:
from transformers import DataCollatorWithPadding

# Creating a data collator that will dynamically pad the inputs received:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## 5. Setting Training Arguments

In [7]:
from transformers import TrainingArguments

# Creating training arguments that contain the model hyperparameters:
training_args = TrainingArguments(
    output_dir="my_bert_model",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    report_to="none",
)


## 6. Model Training



In [8]:
from transformers import Trainer

# Gathering all these classes in Trainer:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [9]:
# Calling train() to start training:
trainer.train()

Step,Training Loss
500,0.542300
1000,0.523200
1500,0.523200
2000,0.489600
2500,0.341200
3000,0.302000
3500,0.310000
4000,0.305400


TrainOutput(global_step=4266, training_loss=0.4081023815814956, metrics={'train_runtime': 261.104, 'train_samples_per_second': 65.338, 'train_steps_per_second': 16.338, 'total_flos': 173971090497408.0, 'train_loss': 0.4081023815814956, 'epoch': 2.0})

## 7. Prediction

In [10]:
# Getting a text for prediction:
text = "I love AI. It's fun to analyze the NLP tasks with Hugging Face"

In [11]:
# Preprocessing the text:
inputs = tokenizer(text, return_tensors="pt")
inputs

{'input_ids': tensor([[  101,  1045,  2293,  9932,  1012,  2009,  1005,  1055,  4569,  2000,
         17908,  1996, 17953,  2361,  8518,  2007, 17662,  2227,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [15]:
import torch
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model_save_path = "my_bert_model.pth"
torch.save(model.state_dict(), model_save_path)

# Calculating predictions:
with torch.no_grad():
    logits = model(**inputs).logits

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# Looking the prediction:
predicted_class_id = logits.argmax().item()
predicted_class_id

0